### Qubo: Basic Set Packing Problem

Given a list of n items, and their weights, select the item with the largest weight.

* Decision variables: $x_i$ for i in (1, 2, ...,n)
* Objective function: $\ max\sum_{i=1}^n w_ix_i \ $ i.e. $\ w_1x_1 + w_2x_2 + ... w_nx_n$
* Constraint: Select exactly one item from the list i.e. $x_1 + x_2 + ... + x_n = 1$

w.r.t. the constraint

* the classical constraint $x+y=1$ can be rewritten in QUBO form as follows: 
$P(1-x-y+2xy)$
* thus in the present case the constraint $x_1 + x_2 + ... + x_n = 1$ can be re-written as: 

    $$P(1 - x_1 - x_2 - ... - x_n + 2x_1x_2 + 2x_1x_3 + ... + 2x_{n-1}x_n)$$
    
    
<br>

**Notes**

* When coding, don't forget that indexing in Python, and consequently also in the *dadk* library, are zero-based!
* Recall that when we maximise a function, we must SUBTRACT the constraint(s) from the Objective Function e.g. $x_1 + x_2 + x_3 + ... - P(1- x_1 - x_2 - ...)$
* In the present example we need to apply a multiplication by -1 on two occasions: first when combining polynomials, then telling *dadk* to find the solution. Whether or not this latter case is needed is, I think, dependant on the form of the penalty term. To be further investigated...


<br>


In [0]:
# import stuff

import itertools
import numpy as np
from dadk.FujitsuEmulatedDASolver import *
emda_solver = FujitsuEmulatedDASolver()

number of GPU available: 0


In [0]:
# Objective function: max x1 + x2 + x3 + x4
def H_obj(items):
    H = BinPol()
    for item in items:
        H.set_term(item[1], (item[0],))
    return H


# Constraint 1: select exactly one item
def H_c1(items, P):
    p = BinPol().set_term(1,())
    for pair in itertools.combinations(items, r=2):
        i = pair[0][0]
        j = pair[1][0]        
        p.set_term(-1,(i,))
        p.set_term(-1,(j,))
    p.multiply(p)
    p.multiply_scalar(P) 
    #print(p)
    return p

def H_all(items, P):
    H = BinPol()
    H.add(H_obj(items))
    H.add(H_c1(items, P).multiply_scalar(-1)) # multiply by -1 to subtract instead of adding
    return H

# Set of four items (itemId, itemWeight)
# Note how ItemId replicates zero-based indexing
items = [(0, 7),(1, 2),(2,5),(3,10)] 

# take a look at the polynomials
print('Objective function to maximise:\n {}'.format(H_obj(items)))
print('Constraint polynomial:\n {}'.format(H_c1(items, P=1)))
print('Subtract constraint polynomial from Objective function:\n {}'
      .format(H_all(items, P=1)))

Objective function to maximise:
 7 x_0 + 2 x_1 + 5 x_2 + 10 x_3
Constraint polynomial:
 1 - 1 x_0 + 2 x_0 x_1 + 2 x_0 x_2 + 2 x_0 x_3 - 1 x_1 + 2 x_1 x_2 + 2 x_1 x_3 - 1 x_2 + 2 x_2 x_3 - 1 x_3
Subtract constraint polynomial from Objective function:
  - 1 + 8 x_0 - 2 x_0 x_1 - 2 x_0 x_2 - 2 x_0 x_3 + 3 x_1 - 2 x_1 x_2 - 2 x_1 x_3 + 6 x_2 - 2 x_2 x_3 + 11 x_3


In [0]:
# Solve It ...

"""try different values of P
note how small values e.g. P=8 do not return a correct answer!
"""
items = [(0, 1),(1, 23),(2,5),(3,10)]

# try different values for P
for P in range(0,14, 2) :
    H = H_all(items, P)#.multiply_scalar(-1)
    # multiply by -1 to maximize instead of minimize
    H.multiply_scalar(-1)
    #print('QUBO Polynomial with penalty applied:\n {}'.format(H))

    emda_sol = emda_solver.minimize(H)
    x = emda_sol.get_minimum_energy_solution().configuration
    cnt_selected = np.count_nonzero(x)
    if not cnt_selected == 1:
        print('P={} -> invalid solution'.format(P))
    else:
        print('P={} -> valid solution: {} value: {}'.format(P, x, H.compute(x)))
#         print("  Solution  %s value %f" % (x, H.compute(x)))

P=0 -> invalid solution
P=2 -> invalid solution
P=4 -> invalid solution
P=6 -> invalid solution
P=8 -> invalid solution
P=10 -> valid solution: [0, 1, 0, 0] value: -23
P=12 -> valid solution: [0, 1, 0, 0] value: -23
